In [1]:
import json
import os
import torch
import torch.nn as nn
import numpy as np
import re
import torch.optim as optim

from transformers import BertTokenizer, BertTokenizerFast, BertForMaskedLM, BertModel, BertConfig,BertForSequenceClassification
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm

## 0. Initialize params

In [2]:
'''
gpu/cpu
'''
cuda_index=3
if torch.cuda.is_available():
    device = torch.device('cuda:'+str(cuda_index))
else:
    device = torch.device('cpu')

'''
modeling
'''

batch_size = 8
shuffle = True

## 1. Load and explore the data

In [3]:
def load_json(path, file):
    with open(os.path.join(path, file), 'r') as json_file:
        data = json.load(json_file)
    return data

In [4]:
'''
read config file
'''
config = load_json('./', 'config.json')
config

{'model_name': 'xlm-roberta-base',
 'train_file': 'Train_Numerical_Reasoning.json',
 'dev_file': 'Dev_Numerical_Reasoning.json',
 'path': '../NumEval - Task 3/'}

In [5]:
path=config['path']
train_file=config['train_file']
dev_file=config['dev_file']

In [6]:
'''
read train and dev files
'''
train_data = load_json(path, train_file)
dev_data = load_json(path, dev_file)

In [7]:
train_data[0]

{'news': "(Oct 7, 2014  12:40 PM CDT) As of Jan. 1, Walmart will no longer offer 30,000 of its employees health insurance. Bloomberg notes that's about 2% of its workforce. The move comes as a reaction to the company's rising health care costs as far more of its employees and their families enrolled in its health care plans than it had expected following the ObamaCare rollout. The AP reports those costs will surge $500 million this fiscal year, $170 million more than had been estimated. Those affected are employees who average fewer than 30 hours of work per week; the Wall Street Journal explains they were grandfathered in when Walmart in 2012 stopped offering insurance to new hires who didn't exceed the 30-hour threshold. A benefits expert says Walmart is actually late to the game in terms of cutting insurance to some part-time workers; Target, the Home Depot, and others have already done so. Meanwhile, Walmart's full time workers will see their premiums rise in 2015. Premiums for the

In [8]:
train_data[1]

{'news': "(Oct 29, 2013  8:15 AM CDT) Dax Shepard and Kristen Bell got married at the Beverly Hills courthouse, in a ceremony about as different from Kim Kardashian's last wedding extravaganza as it is possible to be. As Shepard revealed last night on Jimmy Kimmel Live, the whole thing—including the fuel it took to get to the courthouse—cost $142.  It was just Kristen and I at this lonely courthouse,  he said, so friends showed up afterward with a cake reading, in icing,  The World's Worst Wedding.   How many people can say they threw the world's worst wedding?  Shepard asked.",
 'masked headline': 'Dax Shepard: Wedding to Kristen Bell Cost $____',
 'calculation': 'Copy(142)',
 'ans': '142'}

## 2. Data processing and tokenization

In [9]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

In [10]:
MASK_TOKEN = tokenizer.mask_token
MASK_TOKEN

'[MASK]'

In [11]:
def process_data(sample, replace_token='mask', task='train'):
    '''
    teacher forcing only during training, hence reasoning prompt would be prepended only to the train samples
    '''
    
    news = sample['news']
    masked_headline = sample['masked headline']
    calculation = sample['calculation']
    ans = " " + str(sample['ans']) + " "
    
    if replace_token=='mask':
        replace_token=MASK_TOKEN
    else:
        replace_token=ans
        
    if task=='train':
        input_prompt = "Given the news article, perform " + calculation + " to fill in the mask token : " + "\n" + news + " " + masked_headline.replace('____', replace_token)
    else:
        input_prompt = "Given the news article, fill in the mask token : " + "\n" + news + " " + masked_headline.replace('____', replace_token)
    
    if "copy" in calculation.lower():
        reasoning = 1
    else:
        reasoning = 0
    
    return {"input_prompt":input_prompt, "ans": ans, "reasoning":reasoning}

def tokenize(sentence):
    return tokenizer.encode_plus(sentence,
                                 max_length=512,
                                 padding='max_length',
                                 truncation="only_first",
                                 return_tensors='pt',
                                 return_attention_mask=True)

In [12]:
train_processed = []
dev_processed = []

for i, sample in tqdm(enumerate(train_data)):
    if i==10575:
        continue
    else:
        train_processed.append(process_data(sample, replace_token='ans'))

for i, sample in tqdm(enumerate(dev_data)):
    dev_processed.append(process_data(sample, replace_token='ans', task='dev'))

21157it [00:00, 171651.58it/s]
2572it [00:00, 229034.41it/s]


In [13]:
train_processed[20]

{'input_prompt': "Given the news article, perform Copy(5) to fill in the mask token : \n(Jul 27, 2010  9:52 AM CDT) Previous reports estimated that Chelsea Clinton’s wedding would cost a cool $2 million, but the real number is probably more like $3 million to $5 million. Wedding experts run down the costs for the New York Daily News, from $600,000 air-conditioned tents to the $150-a-pop invitations and $100 place settings for each of the 500 guests. At the more conservative $3 million estimate, the total cost comes to $6,000 per guest. But it’s not all designer dresses, fancy food, and $15,000 port-a-potties (yes, $15,000 for  outhouses  that are much nicer than your bathroom at home—TMZ has pictures). Because of the high-profile nature of the event, security will probably run at least $200,000 (even though the White House confirms President Obama won't attend)—or more if they opt to shut down air space or pay police to monitor traffic. Overcome with Clinton wedding fever? Click here f

In [14]:
dev_processed[4]

{'input_prompt': "Given the news article, fill in the mask token : \n(Oct 16, 2014  3:02 AM CDT) Tristen Kurilla, the Pennsylvania 10-year-old who confessed to killing a 90-year-old woman over the weekend, is still in an adult prison and for now, his family thinks that could be the best place for him. His attorney withdrew a bail request yesterday saying the  family just doesn't feel comfortable for numerous reasons,  including concern for the family of Helen Novak, whom Tristen allegedly killed with a walking stick, reports WBRE. The lawyer also cited the family's worries over  the supervision, of work, of everything.  The district attorney says the boy is not in the  general population. He is not in solitary confinement and he is not in isolation. He is in a cell which I believe is next to the infirmary where he can come and go from the cell to the infirmary.  The nearest juvenile detention facility is 80 miles away. The boy has been provided with coloring books and other forms of en

In [85]:
dev_processed[3]['input_prompt'][50:]

"(Dec 24, 2014  11:19 AM) Turns out you won't even have to leave your house if you want to watch controversial Sony flick The Interview. The company has announced that, as of 1pm Eastern today, the Seth Rogen and James Franco movie about a plot to assassinate Kim Jong Un is available on YouTube, Google Play, Xbox, and possibly Sony's site SeetheInterview.com. It'll cost you $14.99 to buy or $5.99 to rent, Business Insider reports. The movie was initially pulled from release after the Sony hack and ensuing threats, but it will also be released in some theaters tomorrow after all. You Can Watch The Interview at  1 pm"

In [88]:
dev_data[3]

{'news': "(Dec 24, 2014  11:19 AM) Turns out you won't even have to leave your house if you want to watch controversial Sony flick The Interview. The company has announced that, as of 1pm Eastern today, the Seth Rogen and James Franco movie about a plot to assassinate Kim Jong Un is available on YouTube, Google Play, Xbox, and possibly Sony's site SeetheInterview.com. It'll cost you $14.99 to buy or $5.99 to rent, Business Insider reports. The movie was initially pulled from release after the Sony hack and ensuing threats, but it will also be released in some theaters tomorrow after all.",
 'masked headline': 'You Can Watch The Interview at ____pm',
 'calculation': 'Copy(1)',
 'ans': '1'}

In [87]:
dev_data[3]['masked headline']

'You Can Watch The Interview at ____pm'

In [15]:
train_tokenized = [tokenize(sample['input_prompt']) for sample in tqdm(train_processed)]
dev_tokenized = [tokenize(sample['input_prompt']) for sample in tqdm(dev_processed)]

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2572/2572 [00:02<00:00, 1134.22it/s]


In [16]:
'''
get input ids for all the masked token
'''
masked_train_input_ids = [tokenizer(psamples['ans'])['input_ids'][1:-1] for psamples in tqdm(train_processed)]
masked_dev_input_ids = [tokenizer(psamples['ans'])['input_ids'][1:-1] for psamples in tqdm(dev_processed)]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2572/2572 [00:00<00:00, 16037.59it/s]


In [17]:
'''
MLM labels
'''
train_ans_label = torch.stack([inp['input_ids'].clone().detach() for inp in train_tokenized])#.to(device)
dev_ans_label = torch.stack([inp['input_ids'].clone().detach() for inp in dev_tokenized])#.to(device)

In [18]:
def find_last_index_of_continuous_subset(lst, subset):
    subset_length = len(subset)
    
    for i in range(len(lst) - subset_length, -1, -1):
        if lst[i:i + subset_length] == subset:
            return i
    
    # Return -1 if the subset is not found
    return -1

In [19]:
mask_tok_id = tokenizer(MASK_TOKEN)['input_ids'][1:-1][0]
mask_tok_id

103

In [20]:
def replace_with_mask_ids(tokenized_seq, to_mask_ids):
    
    all_masked_inds = []
    
    for i, token in tqdm(enumerate(tokenized_seq)):
        masked_inds = []
        
        total_ls = token['input_ids'][0].tolist()
        sublist = to_mask_ids[i]
    
        last_index_of_subset = find_last_index_of_continuous_subset(total_ls, sublist)
        masked_inds = list(range(last_index_of_subset, last_index_of_subset+len(sublist)))
        all_masked_inds.append(masked_inds)
        
        for ind in masked_inds:
            tokenized_seq[i]['input_ids'][0][ind] = mask_tok_id
        
    return tokenized_seq, all_masked_inds

In [21]:
'''
Model inputs
'''
train_inp, train_masked_inds = replace_with_mask_ids(train_tokenized, masked_train_input_ids)
dev_inp, dev_masked_inds = replace_with_mask_ids(dev_tokenized, masked_dev_input_ids)

21156it [00:01, 15340.95it/s]
2572it [00:00, 12645.75it/s]


In [22]:
train_maxlen = -1
for ids in train_masked_inds:
    train_maxlen = max(train_maxlen, len(ids))
print(train_maxlen)

7


In [23]:
dev_maxlen = -1
for ids in dev_masked_inds:
    dev_maxlen = max(dev_maxlen, len(ids))
print(dev_maxlen)

6


In [24]:
train_masked_inds = torch.stack([torch.tensor(ids+[-1]*(train_maxlen-len(ids))) for ids in train_masked_inds])#.to(device)
dev_masked_inds = torch.stack([torch.tensor(ids+[-1]*(dev_maxlen-len(ids))) for ids in dev_masked_inds])#.to(device)

In [25]:
'''
Reasoning labels
'''
train_reasoning_label = torch.tensor([sample['reasoning'] for sample in tqdm(train_processed)])#.to(device)
dev_reasoning_label = torch.tensor([sample['reasoning'] for sample in tqdm(dev_processed)])#.to(device)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2572/2572 [00:00<00:00, 1339926.70it/s]


In [26]:
train_masked_inds.shape

torch.Size([21156, 7])

In [27]:
dev_masked_inds.shape

torch.Size([2572, 6])

In [28]:
class NumDataset(Dataset):
    
    def __init__(self, input_ids, attn_masks, mask_label, reasoning_label, masked_inds):
        'Initialization'
        self.input_ids = input_ids.squeeze()
        self.attention_mask = attn_masks.squeeze()
        self.mask_labels = mask_label.squeeze().to(torch.float64)
        self.reasoning_label = reasoning_label.squeeze().to(torch.float64)
        self.masked_inds = masked_inds.squeeze()
        
    def __len__(self):
        'Denotes the total number of samples'
        return len(self.input_ids)

    def __getitem__(self, idx):
        'Generates one sample of data'
        return self.input_ids[idx], self.attention_mask[idx], self.mask_labels[idx], self.reasoning_label[idx], self.masked_inds[idx]

def extract_ids_and_masks(inputs):
    
    input_ids = torch.stack([item['input_ids'] for item in inputs]).squeeze(1)
    attention_mask = torch.stack([item['attention_mask'] for item in inputs]).squeeze(1)
    
    return input_ids, attention_mask

In [29]:
train_inp_ids, train_attn_masks = extract_ids_and_masks(train_inp)
dev_inp_ids, dev_attn_masks = extract_ids_and_masks(dev_inp)

In [30]:
params = {
    'batch_size': batch_size,
    'shuffle': False
}

train_set = NumDataset(train_inp_ids, train_attn_masks, train_ans_label, train_reasoning_label, train_masked_inds)
training_generator = torch.utils.data.DataLoader(train_set, **params)

dev_set = NumDataset(dev_inp_ids, dev_attn_masks, dev_ans_label, train_reasoning_label, dev_masked_inds)
dev_generator = torch.utils.data.DataLoader(dev_set, **params)

## 3. Modeling

In [31]:
config = BertConfig.from_pretrained("bert-base-uncased", output_hidden_states=True)

In [32]:
config

BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_hidden_states": true,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.34.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

In [33]:
class BinaryReasoningClassifier(nn.Module):
    """
    2-class classification model : copy, not_copy
    """

    def __init__(self, hidden):
        """
        :param hidden: BERT model output size
        """
        super(BinaryReasoningClassifier, self).__init__()
        # custom layer for binary classification
        self.binary_classification_head = nn.Sequential(
            nn.Linear(config.hidden_size, 128),
            nn.ReLU(),
            nn.Dropout(0.01),
            nn.Linear(128, 2),
            nn.LogSoftmax(dim=-1)
        )
        
    def forward(self, x):
        return self.binary_classification_head(x)

class MaskedPredictor(torch.nn.Module):
    """
    predicting origin token from masked input sequence
    n-class classification problem, n-class = vocab_size
    """

    def __init__(self, d_model, vocab_size):
        """
        :param hidden: output size of BERT model
        :param vocab_size: total vocab size
        """
        super(MaskedPredictor, self).__init__()
        self.linear = nn.Linear(d_model, vocab_size)
        self.softmax = nn.LogSoftmax(dim=-1)

    def forward(self, x):
        return self.softmax(self.linear(x))
    
class NumGenModel(nn.Module):
    
    def __init__(self, config):
        
        super(NumGenModel, self).__init__()
        self.bert = BertModel(config)
        
        self.vocab_size = config.vocab_size
        self.hidden_size = config.hidden_size
        
        self.classifier = BinaryReasoningClassifier(hidden=self.hidden_size)
        self.mask_lm = MaskedPredictor(d_model=self.hidden_size, vocab_size=self.vocab_size)

    def forward(self, input_ids, attention_mask):
        
        # Get the BERT model outputs
        outputs = self.bert(input_ids, attention_mask=attention_mask)
    
        # Extract the pooled output (CLS token) for binary classification
        pooled_output = outputs.pooler_output
        last_hidden = outputs.last_hidden_state
        
        reasoning_preds = self.classifier(pooled_output)
        masked_token_preds = self.mask_lm(last_hidden)
        
        return reasoning_preds, masked_token_preds

In [34]:
def extract_masked_inds(masked_inds):    
    maskinds = []
    for ind in masked_inds:
        if ind==-1:
            break
        else:
            maskinds.append(ind)
    return maskinds

class ScheduledOptim():
    '''A simple wrapper class for learning rate scheduling'''

    def __init__(self, optimizer, d_model, n_warmup_steps):
        self._optimizer = optimizer
        self.n_warmup_steps = n_warmup_steps
        self.n_current_steps = 0
        self.init_lr = np.power(d_model, -0.5)

    def step_and_update_lr(self):
        "Step with the inner optimizer"
        self._update_learning_rate()
        self._optimizer.step()

    def zero_grad(self):
        "Zero out the gradients by the inner optimizer"
        self._optimizer.zero_grad()

    def _get_lr_scale(self):
        return np.min([
            np.power(self.n_current_steps, -0.5),
            np.power(self.n_warmup_steps, -1.5) * self.n_current_steps])

    def _update_learning_rate(self):
        ''' Learning rate scheduling per step '''

        self.n_current_steps += 1
        lr = self.init_lr * self._get_lr_scale()

        for param_group in self._optimizer.param_groups:
            param_group['lr'] = lr

In [40]:
class Trainer:
    def __init__(
        self, 
        model, 
        train_dataloader, 
        dev_dataloader, 
        lr=1e-4,
        weight_decay=0.01,
        betas=(0.9, 0.999),
        warmup_steps=10000,
        log_freq=10
        ):

        self.model = model
        self.train_data = train_dataloader
        self.dev_data = dev_dataloader

        # Setting the Adam optimizer with hyper-param
        self.optim = optim.Adam(self.model.parameters(), lr=lr, betas=betas, weight_decay=weight_decay)
        self.optim_schedule = ScheduledOptim(
            self.optim, config.hidden_size, n_warmup_steps=warmup_steps)

        # Using Negative Log Likelihood Loss function for predicting the masked_token
        self.nllcriterion = torch.nn.NLLLoss(reduction='none')
        self.criterion = nn.CrossEntropyLoss()
        # self.nll_criterion = nn.functional.nll_loss(reduction='none')
        self.log_freq = log_freq
        print("Total Parameters:", sum([p.nelement() for p in self.model.parameters()]))
    
    def train(self, epoch):
        self.iteration(epoch, self.train_data)

    def test(self, epoch):
        self.iteration(epoch, self.dev_data, train=False)

    def iteration(self, epoch, data_loader, train=True):
        
        avg_loss = 0.0
        total_correct = 0
        total_element = 0
        
        mode = "train" if train else "test"
        
        if mode=='train':
            self.model.train()
            
        # progress bar
        data_iter = tqdm(
            enumerate(data_loader),
            desc="EP_%s:%d" % (mode, epoch),
            total=len(data_loader),
            bar_format="{l_bar}{r_bar}"
        )

        for iteration, batch in data_iter:
            if iteration==len(data_iter)-1:
                continue
            inp_ids, attention_mask, mask_label, reasoning_label, masked_inds = batch
            
            inp_ids = inp_ids.to(device)
            attention_mask = attention_mask.to(device)
            mask_label = mask_label.to(device).long()
            reasoning_label = reasoning_label.to(device)
            masked_inds = masked_inds.to(device)
            
            reasoning_pred, token_pred = self.model(input_ids=inp_ids, attention_mask=attention_mask)
            
            # print(f"token_pred = {token_pred.shape}")
            # print(f"label shape = {mask_label.shape}")
            # print(f"reasoning_pred shape = {reasoning_pred.shape}")
            # print(f"reasoning label shape = {reasoning_label.shape}")
            
            # token_pred = token_pred.argmax(axis=-1).to(torch.float64)
            reasoning_pred = reasoning_pred.argmax(axis=1).to(torch.float64)
            
            reasoning_loss = self.criterion(reasoning_pred, reasoning_label)
            
            
            '''
            compute mask
            '''
            minds = [extract_masked_inds(m) for m in masked_inds]
            
            mask = torch.zeros_like(mask_label, dtype=torch.float32)

            # Iterate over the batch and set the mask for each sample
            for i in range(inp_ids.shape[0]):
                mask[i, minds[i]] = 1.0
            
            '''
            compute masked loss
            '''
            
            mlm_loss = nn.functional.nll_loss(token_pred.view(-1, config.vocab_size), mask_label.view(-1), reduction='none')
            
            mlm_loss = mlm_loss.view(batch_size, -1)
            
            mask_loss_numerator = (mlm_loss*mask).sum()
            mask_loss_denominator = mask.sum()
            
            mask_loss = mask_loss_numerator/mask_loss_denominator
            
#             print(f"mask loss numerator = {mask_loss_numerator}")
#             print(f"mask loss denominator = {mask_loss_denominator}")
#             print(f"reasoning loss item = {reasoning_loss.item()}")
            
            loss = reasoning_loss + mask_loss
            # loss.requires_grad=True
            
            # 3. backward and optimization only in train
            if train:
                self.optim_schedule.zero_grad()
                loss.backward()
                self.optim_schedule.step_and_update_lr()

            avg_loss += loss.item()

            post_fix = {
                "epoch": epoch,
                "iter": iteration,
                "avg_loss": avg_loss / (iteration + 1),
                # "avg_acc": total_correct / total_element * 100,
                "loss": loss.item()
            }
            print(post_fix)
            # if i % self.log_freq == 0:
            #     data_iter.write(str(post_fix))
                
            inp_ids = inp_ids.cpu()
            attention_mask = attention_mask.cpu()
            mask_label = mask_label.cpu()
            reasoning_label = reasoning_label.cpu()
            masked_inds = masked_inds.cpu()
        print(
            f"EP{epoch}, {mode}: \
            avg_loss={avg_loss / len(data_iter)}"
        )
        #     total_acc={total_correct * 100.0 / total_element}"
        # ) 

In [41]:
model = NumGenModel(
    config=config
)
model = model.to(device)

In [42]:
trainer = Trainer(model, training_generator, dev_generator)

Total Parameters: 133052348


In [43]:
torch.cuda.empty_cache()

In [44]:
epochs=100
for epoch in range(epochs):
    print(f"Starting epoch = {epoch}")
    trainer.train(epoch)

Starting epoch = 0


EP_train:0:   0%|| 1/1323 [00:00<14:15,  1.54it/s]

{'epoch': 0, 'iter': 0, 'avg_loss': 43.928740548621484, 'loss': 43.928740548621484}


EP_train:0:   0%|| 2/1323 [00:01<13:39,  1.61it/s]

{'epoch': 0, 'iter': 1, 'avg_loss': 47.079874991750145, 'loss': 50.2310094348788}


...

{'epoch': 7, 'iter': 1285, 'avg_loss': 34.540576449230166, 'loss': 35.58170357906752}


EP_train:7:  97%|| 1287/1323 [17:39<00:29,  1.21it/s]

{'epoch': 7, 'iter': 1286, 'avg_loss': 34.54092752210421, 'loss': 34.99240723812514}


EP_train:7:  97%|| 1287/1323 [17:39<00:29,  1.21it/s]


KeyboardInterrupt: 

In [45]:
torch.save(model, 'v_final.pth')

In [41]:
torch.cuda.empty_cache()